# **PROJET TEXT MINING** - *Option 6*
---

Quentin Carric - Xavier Lai - Alexis Morin

## **I - Scrapping**

In [ ]:
from lxml import html
import requests
from bs4 import BeautifulSoup
from urllib.error import HTTPError, URLError
import sys
from progress.bar import IncrementalBar
import json


'''
def get_content(url):
    try:
        resp = requests.get(url)
    except (HTTPError, URLError) as error:
        sys.exit(error)
    return BeautifulSoup(resp.content, "html.parser")

def get_game_url_from_page(url):
    url_game_list = []
    page_content_bs = get_content(url)
    game_tag_a_list = page_content_bs.find_all('a', {"class" : "product-image-link"})
    for game_url in game_tag_a_list:
        url_game_list.append("https://www.micromania.fr" + game_url.attrs['href'])
    return url_game_list


def get_game_infos_dict(game_url):
    page_content_bs = get_content(game_url)
    try:
        game_description = page_content_bs.find('article', {"class" : "product-details__description"}).find('p').get_text()
        game_type = page_content_bs.find('span', {'class' : 'attribute-value product-details__atr-value col-7 col-md-8'}).get_text()
    except AttributeError:
        return {"description" : None, "type" : None}
    return {"description" : game_description, "type" : game_type}


def extrat_all_games(number_page):
    start_value = 0
    desc_and_type_list = []
    for i in range(1,number_page):
        url = f'https://www.micromania.fr/jeux.html?start={start_value}&sz=48&page={i}'
        game_url_list = get_game_url_from_page(url)
        bar = IncrementalBar('scrap video games', max = len(game_url_list))
        print(f"\npage {i}") 
        for game_url in game_url_list:
            bar.next()
            
            desc_and_type_list.append(get_game_infos_dict(game_url))
        start_value += 48
        bar.finish()
    return desc_and_type_list

result = extrat_all_games(30)
with open('data_projet_textmining', 'w') as file:
    json.dump(result, file)
'''

On convertit ensuite notre fichier *json* en un dataframe pandas en regroupant les modalités de notre **Y**.

**Y** aura donc 3 modalités : "Sport et conduite", "Action & Guerre" et "Autres".

In [ ]:
!pip install progress

  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8075 sha256=90e44487c7de44b7762bd3d141fc289b5a1bb6e6e0177729a7458e997ea6f4b8
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built progress


In [ ]:
import json
import pandas as pd
from lxml import html
import requests
from bs4 import BeautifulSoup
from urllib.error import HTTPError, URLError
import sys
from progress.bar import IncrementalBar
import json



dico_type = {
    "Action / aventure" :"Action & Guerre",
    "Jeu de rôle" : "Autres",
    "Sport" : "Sport & Conduite",
    "Football" : "Sport & Conduite",
    "Danse / musique" : "Sport & Conduite",
    "Action" : "Action & Guerre",
    "Course / arcade" : "Sport & Conduite",
    "Gestion" : "Autres",
    "Wargame" : "Action & Guerre",
    "Baston / combat" : "Action & Guerre",
    "Activités récréatives" : "Autres",
    "Plate-forme" : "Autres",
    "Tir" : "Action & Guerre",
    "Console" : "Autres",
    "Aventure" : "Action & Guerre",
    "Stratégie temps réel" : "Action & Guerre",
    "Infiltration" : "Action & Guerre",
    "Rallye" : "Sport & Conduite",
    "Basket" : "Sport & Conduite",
    "Réflexion" : "Autres",
    "Quake-like" : "Action & Guerre",
    "Action / réflexion" : "Action & Guerre",
    "Accessoire" : "Autres",
    "Beat'em all" : "Action & Guerre",
    "Shoot'em up" : "Action & Guerre",
    "Course / simulation" : "Sport & Conduite",
    "Simulation de vol" : "Sport & Conduite",
    "Shoot tactique" : "Action & Guerre",
}

description_list=[]
type_list=[]
with open("data_projet_textmining.json") as json_file:
    data = json.load(json_file)
    description_list = []
    for game_dict in data :
        data_is_valid = (
            game_dict['type'] is not None and 
            game_dict['description'].strip() !='' and 
            game_dict['description'] not in description_list
            )
        if data_is_valid:
            description_list.append(game_dict['description'].strip())
            type_list.append(game_dict['type'].replace('\n', "").strip())

df = pd.DataFrame({'description': description_list, 'type': type_list})
df = df.replace(dico_type)
df["type"] = df["type"].astype("category")
print(df)
print(df.groupby('type').nunique())


                                           description              type
0    Bowser a encore fait des siennes... et cette f...   Action & Guerre
1    Le jeu de tir de référence est de retour avec ...            Autres
2    Vous avez toujours une bonne excuse pour ne pa...  Sport & Conduite
3    Rassemblez-vous et faites passer le message : ...  Sport & Conduite
4    Véritable phénomène international, la saga ...   Action & Guerre
..                                                 ...               ...
601  Smart Four Connected est un jeu de puzzle élec...            Autres
602  Pour la 1ere fois en France, découvrez le stre...  Sport & Conduite
603  « Les légendes éorzéennes racontent que lorsqu...            Autres
604  L’édition spéciale d’Harvest Moon : Lumière d’...            Autres
605  Dans cette nouvelle aventure, Dr. Eggman, touj...            Autres

[606 rows x 2 columns]
                  description
type                         
Action & Guerre           289
Autres    

Nous avons scrappé les descriptions des jeux vidéos ainsi que la catégorie dans laquelle ils appartiennent (source : https://www.micromania.fr/jeux.html)

Nous avons donc en *features* les descriptions , et la variable à expliquer est la catégorie du jeu. Le but est de construire un algortihme qui permettra de prédire la catégorie d'un jeu en regardant uniquement sa description.

## **II - Traitements de données**

In [ ]:
!pip install nltk

In [ ]:
import numpy as np
import pandas as pd
from string import punctuation
import nltk
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS 
import pickle
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import scipy.sparse

nltk.download('punkt')
nltk.download('stopwords')

# local functions

def remove_ponctuation(s):
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')
    return s

def identify_tokens(row):
    review = row
    tokens = nltk.word_tokenize(review)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

def stem_list(row):
    my_list = row
    stemmed_list = [stemming.stem(word) for word in my_list]
    return (stemmed_list)

def remove_stops(row):
    my_list = row
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

def rejoin_words(row):
    my_list = row
    joined_words = ( " ".join(my_list))
    return joined_words

def export_predictions(nom,pred):
    dtest = pd.read_json(Path_data+"test.json")
    dtest['Category'] = pred
    soumission = dtest[["Id","Category"]]
    soumission.to_csv(nom+".csv", index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Mise en place de la validation croisée : Méthode Hold out

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2,random_state = 42)
X_train = train.description
X_test = test.description
Y_train = train.type
Y_test = test.type

In [ ]:
## Traitement de texte

# A) Lower
X_train = X_train.str.lower()
X_test = X_test.str.lower()

# B) Remove ponctuation
X_train = X_train.map(remove_ponctuation)
X_test = X_test.map(remove_ponctuation)

# C) Tokenisation+ stemmisation
X_train = X_train.apply(identify_tokens)
stemming = PorterStemmer()
X_train = X_train.apply(stem_list)

X_test = X_test.apply(identify_tokens)
X_test = X_test.apply(stem_list)

# D) Keep only meaningful stem and join
stops = set(stopwords.words("french")) 
X_train = X_train.apply(remove_stops)
X_train = X_train.apply(rejoin_words)

X_test = X_test.apply(remove_stops)
X_test = X_test.apply(rejoin_words)

## **III - Featuring (TF-ID)**

In [ ]:
transformer = TfidfVectorizer().fit(X_train.values)
print("NB features: %d" %(len(transformer.vocabulary_)))
x_train_tfID = transformer.transform(X_train.values)
x_test_tfID = transformer.transform(X_test.values)

NB features: 5133


## **IV - Learning**

### A) Régression logistique multinomiale

In [ ]:
logistic = LogisticRegression()
logistic.fit(x_train_tfID, Y_train)
predictions = logistic.predict(x_test_tfID)
predictions[:10]

array(['Autres', 'Action & Guerre', 'Action & Guerre', 'Action & Guerre',
       'Action & Guerre', 'Action & Guerre', 'Action & Guerre',
       'Action & Guerre', 'Action & Guerre', 'Action & Guerre'],
      dtype=object)

In [ ]:
logistic.score(x_test_tfID,Y_test)

0.5819672131147541

### B) Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def gridSearch_Pipeline(algo_pipeline,dico_params, nb_cv = 5,x_train = x_train_tfID,y_train=Y_train, x_test=x_test_tfID, y_test=Y_test):
    grid = GridSearchCV(algo_pipeline,dico_params,cv = nb_cv)
    #grid.get_params() pour voir les paramètres possibles
    grid.fit(x_train,y_train)
    print("\n","-"*40,"\nBest parameter (CV score=%0.3f):"%grid.best_score_)
    print(grid.best_params_)
    print("{}\nEstimation de l'Accuracy :\n{}".\
          format('*'*30,grid.score(x_test,y_test)))
    

    pred_algo = grid.predict_proba(x_test)[:,1]
    accuracy_algo = grid.score(x_test,y_test).round(3)
    
    return grid,pred_algo,accuracy_algo

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline

RF_pipeline = make_pipeline(
    RandomForestClassifier(random_state=0,n_jobs = -1))

mtry = np.arange(1500,2500,25)
params={
        "randomforestclassifier__max_features": mtry,
        }

grid, pred_rf, accur_rf = gridSearch_Pipeline(RF_pipeline,params)



 ---------------------------------------- 
Best parameter (CV score=0.562):
{'randomforestclassifier__max_features': 1850}
******************************
Estimation de l'Accuracy :
0.5573770491803278
